# Preliminares

Esta rede irá gerar uma lista de notas (tons em MIDI). Os dados do treinamento são do projeto Magenta, em particular a base do Bach Doodle.

São utilizadas redes LSTM para gerar as sequencias de notas que possuem tom e tempo.

In [ ]:
# conecta com o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports
Caso alguma das células falhe, favor verificar os scripts de atualização/instalação de dependências presente na última seção "Utilidades".

In [ ]:
# importa as dependências gerais
import os
from google.colab import files

In [ ]:
# importa o tensorflow
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity('DEBUG')

In [ ]:
# importa as dependências do magenta
import magenta
import note_seq

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training

HParams = contrib_training.HParams

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from magenta.models.music_vae import data
from magenta.models.music_vae import Config
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import music_vae_train
from magenta.models.music_vae.base_model import MusicVAE
from magenta.models.music_vae.trained_model import TrainedModel

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

# Declaração de constantes

In [ ]:
EPOCHS = 1
SHARDS_LEN = 10
BATCH_SIZE = 2048
OUTPUT_PATH = 'training_dir/data/'

current_step = 0

# Definição das funções de prepação dos lotes

In [ ]:
def get_train_batch_path(index):
  return OUTPUT_PATH + "batch-3-%i.tfrecord"%(index)

In [ ]:
def prepare_training_set(dataset_path):
  tr_raw_set = tf.data.TFRecordDataset(dataset_path) # 44389 registros
  tr_data = tr_raw_set.shuffle(BATCH_SIZE * SHARDS_LEN)

  num_shards = SHARDS_LEN

  if not os.path.exists(OUTPUT_PATH):
      os.makedirs(OUTPUT_PATH)

  for i in range(0, SHARDS_LEN):
    writer = tf.data.experimental.TFRecordWriter(get_train_batch_path(i))
    writer.write(tr_data.shard(SHARDS_LEN, i))

# Construção dos modelos

In [ ]:
model = MusicVAE(lstm_models.BidirectionalLstmEncoder(), 
                 lstm_models.CategoricalLstmDecoder())

hparams = merge_hparams(
    lstm_models.get_default_hparams(),
    HParams(
        batch_size=BATCH_SIZE,
        max_seq_len=32,  # Divide em 2 compassos com 16 batidas cada
        z_size=512,
        enc_rnn_size=[2048],
        dec_rnn_size=[2048, 2048, 2048],
        free_bits=0,
        max_beta=0.5,
        beta_rate=0.99999,
        learning_rate=0.005,
        sampling_schedule='inverse_sigmoid',
        sampling_rate=1000,
        ))

augmenter = data.NoteSequenceAugmenter(transpose_range=(-5, 5)),

data_converter = data.OneHotMelodyConverter(
    skip_polyphony=False,
    max_bars=100,
    slice_bars=2,
    steps_per_quarter=4)

In [ ]:
#model.build(hparams=hparams, output_depth=data_converter.output_depth, is_training=True)

In [ ]:
def get_model_config():
  global current_step
  return Config(
    model=model,
    hparams=hparams,
    #note_sequence_augmenter=augmenter,
    data_converter=data_converter,
    train_examples_path=get_train_batch_path(current_step),
  )

# Definição das funções de treinamento

In [ ]:
def get_dataset(tf_file_reader=tf.data.TFRecordDataset):
  return data.get_dataset(
      get_model_config(),
      tf_file_reader=tf_file_reader,
      is_training=True,
      cache_dataset=False)

In [ ]:
def train_step():
    run_dir = os.path.expanduser('training_dir')
    train_dir = os.path.join(run_dir, 'train')
    config = get_model_config()
    music_vae_train.train(
        train_dir=train_dir,
        config=config,
        dataset_fn=get_dataset,
        num_steps=hparams.batch_size,
        checkpoints_to_keep=1,
        task=0
    )

In [ ]:
def train():
  global current_step

  for i in range(0, EPOCHS):
    current_step = 0;
    for j in range (0, SHARDS_LEN):
      try:
        train_step()
        current_step += 1
      except:
        current_step += 1
        pass

# Executáveis

In [ ]:
# monta os lotes de treinamento
prepare_training_set('drive/MyDrive/BachDoodle/005.tfrecord')

In [ ]:
# execute para treinar o modelo
# a partir desse momento não é mais possível preparar os lotes (devido disable_v2_behavior)
tf.disable_v2_behavior()
train()

In [ ]:
# instancia o modelo treinado a partir do último checkpoint
# OBS: Em caso de erro no deepCopy, favor alterar linha 58 do
#      trained_model.py do MusicVAE para self._config = config
#      e reiniciar o ambiente de execução
trained_model = TrainedModel(
    config=get_model_config(), 
    batch_size=4, 
    checkpoint_dir_or_path='/content/training_dir/train/model.ckpt-0')

In [ ]:
# executa o gerador treinado
generated_sequences = trained_model.sample(n=4, length=90, temperature=1.0)

for ns in generated_sequences:
  # print(ns)
  note_seq.plot_sequence(ns)

# **Utilidades...**

In [ ]:
# instala o magenta
!pip install -qU magenta

In [ ]:
# printa a versão do python
!python --version

In [ ]:
# cria o arquivo de requerimentos
!pip freeze >> requierements.txt 

In [ ]:
# cria um .zip dos dados de treinamento
import shutil
shutil.make_archive('training_dir/train/', 'zip', 'training_dir/train/')

'/content/training_dir/train.zip'

In [ ]:
# copia conteúdos do .zip para o drive
!cp training_dir/train.zip /content/drive/MyDrive/LSTM-2

In [ ]:
# conta a quantidade de registros existentes no dataset de treinamento
sum(1 for _ in tf.data.TFRecordDataset('training_dir/data/batch-0.tfrecord'))

In [ ]:
# inspeciona o lote de dados
tr_raw_set = tf.data.TFRecordDataset('training_dir/data/batch-0.tfrecord')
for raw_record in tr_raw_set.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)